# Data parsing

In [ ]:
import json
from func_to_call import parse_all_data, parse_data_with_time


data_val = parse_all_data('val_set.json')
data_train = parse_all_data('train_set.json')

with open('parsed_tuning_val.json', 'w', encoding='utf-8') as f:
    json.dump(data_val, f, ensure_ascii=False)
with open('parsed_tuning_train.json', 'w', encoding='utf-8') as f:
    json.dump(data_train, f, ensure_ascii=False)

In [ ]:
import json

with open('parsed_tuning_train.json', 'r', encoding='utf-8') as f:
    training_data = json.load(f)

In [ ]:
formatted_data = []

for item in training_data:
    contexts = "\n".join([ctx['text'] for ctx in item['contexts']])
    base_input = f"Вопрос: {item['user_question']}\nКонтекст: {contexts}"

    if item['winner'] == 'Saiga':
        formatted_data.append({
            "input": base_input,
            "output": item['saiga_answer'],
            "source": "saiga",
            "rating": "good" if item['winner'] in ['Saiga', 'Оба хорошо'] else "bad"
        })

    elif item['winner'] == 'GigaChat':
        formatted_data.append({
            "input": base_input,
            "output": item['giga_answer'],
            "source": "giga",
            "rating": "good" if item['winner'] in ['GigaChat', 'Оба хорошо'] else "bad"
        })

    elif item['winner'] == 'Оба хорошо':
        formatted_data.extend([
            {
                "input": base_input,
                "output": item['saiga_answer'],
                "source": "saiga",
                "rating": "good"
            },
            {
                "input": base_input,
                "output": item['giga_answer'],
                "source": "giga",
                "rating": "good"
            }
        ])

    elif item['winner'] == 'Оба плохо':
        formatted_data.extend([
            {
                "input": base_input,
                "output": item['saiga_answer'],
                "source": "saiga",
                "rating": "bad"
            },
            {
                "input": base_input,
                "output": item['giga_answer'],
                "source": "giga",
                "rating": "bad"
            }
        ])

    else:
        formatted_data.append({
            "input": base_input,
            "output": item['saiga_answer'],
            "source": "unknown",
            "rating": "neutral"
        })

#Model tuning

In [ ]:
pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# Load t5
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from flax import linen as nn

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
last_layer = model.lm_head
count = 0
for param in model.parameters():
  count+=1
count

190

In [ ]:
class LoRALMHead(nn.Module):
  original_layer: nn.Dense
  rank: int = 8

  def setup(self):
    self.lora_A = nn.Dense(self.rank, use_bias=False)
    self.lora_B = nn.Dense(self.original_layer.features, use_bias=False)

  def __call__(self, hidden_states):
    return self.original_layer(hidden_states) + self.lora_B(self.lora_A(hidden_state))

In [ ]:
lora_ll_model = LoRALMHead(original_layer=last_layer, rank=8)
lora_ll_model.

ValueError: Can't access variables on unbound modules

#Training